# ANOVA  - Lab

## Introduction

In this lab, you'll get some brief practice generating an ANOVA table (AOV) and interpreting its output. You'll also perform some investigations to compare the method to the t-tests you previously employed to conduct hypothesis testing.

## Objectives

In this lab you will: 

- Use ANOVA for testing multiple pairwise comparisons 
- Interpret results of an ANOVA and compare them to a t-test

## Load the data

Start by loading in the data stored in the file `'ToothGrowth.csv'`: 

In [1]:
import pandas as pd

# Your code here
# Load the data
df = pd.read_csv('ToothGrowth.csv')
df.head()

,len,supp,dose
0,4.2,VC,0.5
1,11.5,VC,0.5
2,7.3,VC,0.5
3,5.8,VC,0.5
4,6.4,VC,0.5


## Generate the ANOVA table

Now generate an ANOVA table in order to analyze the influence of the medication and dosage:  

In [2]:
from statsmodels.formula.api import ols

# Your code here
import statsmodels.api as sm

# Perform ANOVA
model = ols('len ~ C(supp) + C(dose) + C(supp):C(dose)', data=df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

                      sum_sq    df          F        PR(>F)
C(supp)           205.350000   1.0  15.571979  2.311828e-04
C(dose)          2426.434333   2.0  91.999965  4.046291e-18
C(supp):C(dose)   108.319000   2.0   4.106991  2.186027e-02
Residual          712.106000  54.0        NaN           NaN


## Interpret the output

Make a brief comment regarding the statistics and the effect of supplement and dosage on tooth length: 

1. Conclusion:
Both the type of supplement and dosage have significant effects on tooth length, and there is also a significant interaction between the two. Therefore, the impact of the supplement on tooth length may vary depending on the dosage.

## Compare to t-tests

Now that you've had a chance to generate an ANOVA table, its interesting to compare the results to those from the t-tests you were working with earlier. With that, start by breaking the data into two samples: those given the OJ supplement, and those given the VC supplement. Afterward, you'll conduct a t-test to compare the tooth length of these two different samples: 

In [3]:
from scipy.stats import ttest_ind

# Your code here
# Split the data into two samples based on the supplement type
oj_sample = df[df['supp'] == 'OJ']['len']
vc_sample = df[df['supp'] == 'VC']['len']

# Conduct a t-test to compare the tooth length of the two samples

t_stat, p_val = ttest_ind(oj_sample, vc_sample)
print(f"T-statistic: {t_stat}, P-value: {p_val}")

T-statistic: 1.91526826869527, P-value: 0.06039337122412849


Now run a t-test between these two groups and print the associated two-sided p-value: 

In [5]:
# Calculate the 2-sided p-value for a t-test comparing the two supplement groups
p_val = p_val / 2
# BEGIN: t-test for the two supplement groups
t_stat, p_val = ttest_ind(oj_sample, vc_sample)
print(f"T-statistic: {t_stat}, P-value: {p_val}")
# END:
print(f"T-statistic: {t_stat}, P-value: {p_val}")


T-statistic: 1.91526826869527, P-value: 0.06039337122412849
T-statistic: 1.91526826869527, P-value: 0.06039337122412849


## A 2-Category ANOVA F-test is equivalent to a 2-tailed t-test!

Now, recalculate an ANOVA F-test with only the supplement variable. An ANOVA F-test between two categories is the same as performing a 2-tailed t-test! So, the p-value in the table should be identical to your calculation above.

> Note: there may be a small fractional difference (>0.001) between the two values due to a rounding error between implementations. 

In [6]:
# Perform ANOVA with only the supplement variable
model_supp = ols('len ~ C(supp)', data=df).fit()
anova_table_supp = sm.stats.anova_lm(model_supp, typ=2)
print(anova_table_supp)

# Extract the p-value for the supplement variable
anova_p_val = anova_table_supp.loc['C(supp)', 'PR(>F)']
print(f"ANOVA p-value: {anova_p_val}")
print(f"T-test p-value: {p_val}")

               sum_sq    df         F    PR(>F)
C(supp)    205.350000   1.0  3.668253  0.060393
Residual  3246.859333  58.0       NaN       NaN
ANOVA p-value: 0.060393371224128745
T-test p-value: 0.06039337122412849


## Run multiple t-tests

While the 2-category ANOVA test is identical to a 2-tailed t-test, performing multiple t-tests leads to the multiple comparisons problem. To investigate this, look at the various sample groups you could create from the 2 features: 

In [7]:
for group in df.groupby(['supp', 'dose'])['len']:
    group_name = group[0]
    data = group[1]
    print(group_name)

('OJ', 0.5)
('OJ', 1.0)
('OJ', 2.0)
('VC', 0.5)
('VC', 1.0)
('VC', 2.0)


While bad practice, examine the effects of calculating multiple t-tests with the various combinations of these. To do this, generate all combinations of the above groups. For each pairwise combination, calculate the p-value of a 2-sided t-test. Print the group combinations and their associated p-value for the two-sided t-test.

In [8]:
from itertools import combinations

# Your code here; reuse your t-test code above to calculate the p-value for a 2-sided t-test
# for all combinations of the supplement-dose groups listed above. 
# (Since there isn't a control group, compare each group to every other group.)
# Generate all combinations of the supplement-dose groups
grouped = df.groupby(['supp', 'dose'])['len']
group_combinations = list(combinations(grouped, 2))

# Calculate the p-value for a 2-sided t-test for each pairwise combination
for (name1, group1), (name2, group2) in group_combinations:
    t_stat, p_val = ttest_ind(group1, group2)
    print(f"Comparison: {name1} vs {name2}, P-value: {p_val}")

Comparison: ('OJ', 0.5) vs ('OJ', 1.0), P-value: 8.357559281443774e-05
Comparison: ('OJ', 0.5) vs ('OJ', 2.0), P-value: 3.4018585295016214e-07
Comparison: ('OJ', 0.5) vs ('VC', 0.5), P-value: 0.005303661339923052
Comparison: ('OJ', 0.5) vs ('VC', 1.0), P-value: 0.04223992429368205
Comparison: ('OJ', 0.5) vs ('VC', 2.0), P-value: 7.025409196997986e-06
Comparison: ('OJ', 1.0) vs ('OJ', 2.0), P-value: 0.03736279585664383
Comparison: ('OJ', 1.0) vs ('VC', 0.5), P-value: 1.3372624230559434e-08
Comparison: ('OJ', 1.0) vs ('VC', 1.0), P-value: 0.0007807261651774468
Comparison: ('OJ', 1.0) vs ('VC', 2.0), P-value: 0.09583711277517494
Comparison: ('OJ', 2.0) vs ('VC', 0.5), P-value: 1.3381068810881244e-11
Comparison: ('OJ', 2.0) vs ('VC', 1.0), P-value: 2.3131084633597503e-07
Comparison: ('OJ', 2.0) vs ('VC', 2.0), P-value: 0.9637097790041267
Comparison: ('VC', 0.5) vs ('VC', 1.0), P-value: 6.492264598157612e-07
Comparison: ('VC', 0.5) vs ('VC', 2.0), P-value: 4.957285658438862e-09
Comparison: 

## Summary

In this lesson, you implemented the ANOVA technique to generalize testing methods to multiple groups and factors.